In [1]:
# import libraries 
import ee
import geemap
import json
import os
import requests
import geopandas as gpd
import pandas as pd
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON

In [2]:
file_path = os.path.abspath('../Datasets/telangana_shapefile.geojson')

if not os.path.exists(file_path):
    url = 'https://github.com/undpindia/Data4Policy/tree/main/Geospatial%20Data%20Science%20Internship/Datasets/telangana_shapefile.geojson'
    r = requests.get(url)
    with open(file_path, 'w') as f:
        f.write(r.content.decode("utf-8"))

with open(file_path) as f:
    json_data = json.load(f)
    tel_gpd = gpd.read_file(file_path)

In [3]:
tel_gpd['centroid'] = tel_gpd.centroid 
tel_gpd.centroid
#tel_gpd = tel_gpd.set_geometry('centroid_column')

C:\Users\emeka\AppData\Local\Temp\ipykernel_23532\4120886396.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  tel_gpd['centroid'] = tel_gpd.centroid
C:\Users\emeka\AppData\Local\Temp\ipykernel_23532\4120886396.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  tel_gpd.centroid


0    POINT (79.05946 17.79484)
dtype: geometry

In [4]:
Map = geemap.Map(center=[17.79484, 79.05946], zoom = 7)

In [5]:
json_layer = GeoJSON(
    data=json_data,
    name='Telangana',
    hover_style={'fillColor': 'red', 'fillOpacity': 0.5},
)
Map.add_layer(json_layer)

In [6]:
ee_data = geojson_to_ee(json_data)
#Map.addLayer(ee_data, {}, "Telangana")

In [7]:
Map

Map(center=[17.79484, 79.05946], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

In [8]:
lulc= ee.ImageCollection('ESA/WorldCover/v100')\
        .filterBounds(ee_data) \
        .filterDate('2020-01-01', '2021-01-01') \
        .mean() \
        .clip(clip_geometry=ee_data)
lulc_unbound = ee.ImageCollection('ESA/WorldCover/v100').mean()

dict = {
  "names": [
    "Water",
    "Trees",
    "Grass",
    "Flooded Vegetation",
    "Crops",
    "Scrub/Shrub",
    "Built Area",
    "Bare Ground",
    "Snow/Ice",
    "Clouds"
  ],
  "colors": [
    "#1A5BAB",
    "#358221",
    "#A7D282",
    "#87D19E",
    "#FFDB5C",
    "#EECFA8",
    "#ED022A",
    "#EDE9E4",
    "#F2FAFF",
    "#C8C8C8"
  ]}

In [9]:
dict['colors']
palette = []
for i in dict['colors']:
    palette.append(i)
    
palette
#visualization = {
#  min:1, max:10, palette:palette
#}

['#1A5BAB',
 '#358221',
 '#A7D282',
 '#87D19E',
 '#FFDB5C',
 '#EECFA8',
 '#ED022A',
 '#EDE9E4',
 '#F2FAFF',
 '#C8C8C8']

In [10]:
geom = ee_data.geometry()

In [11]:
Map.addLayer(lulc, {}, 'Land Use Land Cover')

In [12]:
out_dir = '../Datasets'
filename = os.path.join(out_dir, 'lulc.tif')
geemap.ee_export_image(
    lulc, filename=filename, scale=220, region=geom
)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\emeka\Documents\GitHub\Data4Policy\Geospatial Data Science Internship\Datasets\lulc.tif


In [13]:
geemap.ee_export_image_to_drive(
  lulc,
  description= 'LULC',
  scale= 10, 
  folder='./backup',
  region= ee_data
)

Exporting LULC ...


In [14]:
lulc_vector = lulc_unbound.reduceToVectors(scale=240, maxPixels=10000000, geometry = geom)

In [15]:
geemap.ee_to_shp(lulc_vector, filename='../Datasets/lulc.shp')

Image.reduceToVectors: First band ('Map') of image must be integral.


In [16]:
fire_path = os.path.abspath('../Datasets/fireIncident.geojson')
with open(fire_path) as fr:
    fire_data = json.load(fr)

In [17]:
fireI_layer = GeoJSON(
    data=fire_data,
    name='Fire Incident',
    hover_style={'fillColor': 'green', 'fillOpacity': 0.5},
)

In [18]:
ee_fire = geojson_to_ee(fire_data)

Since the size of the LULC is large, part of the pre-processing, i.e., mask extraction, mosaicing, aggregation, and spatial join between the fire Incident dataset and the LULC was done using QGIS.

In [19]:
fire_lulc = gpd.read_file('../Datasets/fire_lulc.shp')
fire_lulc.info()
fire_lulc.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 18785 entries, 0 to 18784
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   FID_1       18785 non-null  int64   
 1   OBJECTID    18785 non-null  int64   
 2   latitude    18785 non-null  float64 
 3   longitude   18785 non-null  float64 
 4   brightness  18785 non-null  float64 
 5   scan        18785 non-null  float64 
 6   track       18785 non-null  float64 
 7   acq_date    18785 non-null  object  
 8   acq_time    18785 non-null  int64   
 9   satellite   18785 non-null  object  
 10  instrument  18785 non-null  object  
 11  confidence  18785 non-null  int64   
 12  version     18785 non-null  float64 
 13  bright_t31  18785 non-null  float64 
 14  frp         18785 non-null  float64 
 15  daynight    18785 non-null  object  
 16  type        18785 non-null  int64   
 17  year        18785 non-null  int64   
 18  FID_2       18785 non-null  int64   
 

,FID_1,OBJECTID,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,...,frp,daynight,type,year,FID_2,gridcode,COUNT_grid,area,lulc,geometry
0,59,60,17.1877,80.2715,311.9,1.0,1.0,2015-01-14T00:00:00,521,Terra,...,5.4,D,0,2015,0,1,25196,0.0,Trees,POINT Z (422531.972 1900465.076 0.000)
1,96,97,18.5925,77.8659,311.9,1.1,1.1,2015-01-19T00:00:00,539,Terra,...,6.6,D,0,2015,0,1,25196,0.0,Trees,POINT Z (169209.327 2058626.064 0.000)
2,124,125,18.7582,80.0033,312.6,3.4,1.7,2015-01-24T00:00:00,557,Terra,...,39.7,D,0,2015,0,1,25196,0.0,Trees,POINT Z (394942.608 2074367.031 0.000)
3,125,126,18.7644,79.9724,313.1,3.3,1.7,2015-01-24T00:00:00,557,Terra,...,42.7,D,0,2015,0,1,25196,0.0,Trees,POINT Z (391689.269 2075071.624 0.000)
4,126,127,18.7629,79.9818,316.2,3.6,1.8,2015-01-24T00:00:00,726,Aqua,...,56.6,D,0,2015,0,1,25196,0.0,Trees,POINT Z (392679.179 2074899.944 0.000)


### Preamble


- First, from prior knowledge as well as the_info class_, it is important to note that several of the features/predictors from their definitions are not good variable to determine the objective of the task - if the fires were agricultural or not. They include:  state instrument used, that satellite track path, satellite scan (instrument scan),  version, type, year, FID_2, gridcode, count_grid, area, OBJECTID, FID_1, geometry
- From all investigation and research, agricultural planting and harvesting varies depending on the crop to be planted. Generally, most planting and harvesting happen in the between April and July and between September and mid December.


In [20]:
fire_lulc_df = fire_lulc.drop(labels=['scan', 'track', 'satellite', 'instrument', 'latitude', 'longitude', 'version', 'type', 'year', 'FID_2', 'gridcode', 'COUNT_grid', 'area', 'OBJECTID', 'FID_1', 'geometry'], axis =1)

In [21]:
#fire_lulc_df['acq_date'] = pd.to_datetime(fire_lulc_df['acq_date'], format='%Y%m%d %I:%M%p)
fire_lulc_df['month'] = pd.DatetimeIndex(fire_lulc_df['acq_date']).month

In [22]:
fire_lulc_df['period_day'] = pd.cut(fire_lulc_df['acq_time'], bins=[0, 500, 1200, 1700, 2100, 2400], include_lowest=True, labels=['night', 'morning', 'afternoon', 'evening', 'night'], ordered=False)

In [23]:
fire_lulc_df.drop(["acq_time", "acq_date"], axis=1)

,brightness,confidence,bright_t31,frp,daynight,lulc,month,period_day
0,311.9,37,301.0,5.4,D,Trees,1,morning
1,311.9,40,296.3,6.6,D,Trees,1,morning
2,312.6,0,300.4,39.7,D,Trees,1,morning
3,313.1,32,300.6,42.7,D,Trees,1,morning
4,316.2,39,303.2,56.6,D,Trees,1,morning
...,...,...,...,...,...,...,...,...
18780,316.8,49,295.2,10.5,D,Moss and lichen,8,morning
18781,321.6,0,299.3,9.4,D,Moss and lichen,8,morning
18782,316.4,55,295.0,11.3,D,Moss and lichen,8,morning
18783,318.1,38,298.3,8.3,D,Moss and lichen,8,morning


In [24]:
#boolean field representing either period of planting and harvesting harvesting/planting = Yes/1 and No/0

def getagricPeriod(month):
    if (month == 4 or month == 5 or month == 6 or month == 7):
       return 'Yes'
    elif(month == 9 or month == 10 or month == 11 or month == 12):
       return 'Yes'
    else:
       return 'No'

fire_lulc_df['plant_harvest'] = fire_lulc_df['month'].apply(getagricPeriod)

In [25]:
fire_lulc_df

,brightness,acq_date,acq_time,confidence,bright_t31,frp,daynight,lulc,month,period_day,plant_harvest
0,311.9,2015-01-14T00:00:00,521,37,301.0,5.4,D,Trees,1,morning,No
1,311.9,2015-01-19T00:00:00,539,40,296.3,6.6,D,Trees,1,morning,No
2,312.6,2015-01-24T00:00:00,557,0,300.4,39.7,D,Trees,1,morning,No
3,313.1,2015-01-24T00:00:00,557,32,300.6,42.7,D,Trees,1,morning,No
4,316.2,2015-01-24T00:00:00,726,39,303.2,56.6,D,Trees,1,morning,No
...,...,...,...,...,...,...,...,...,...,...,...
18780,316.8,2021-08-12T00:00:00,504,49,295.2,10.5,D,Moss and lichen,8,morning,No
18781,321.6,2021-08-12T00:00:00,505,0,299.3,9.4,D,Moss and lichen,8,morning,No
18782,316.4,2021-08-15T00:00:00,535,55,295.0,11.3,D,Moss and lichen,8,morning,No
18783,318.1,2021-08-24T00:00:00,529,38,298.3,8.3,D,Moss and lichen,8,morning,No


In [26]:
fire_lulc_df = fire_lulc_df.drop(["acq_time", "acq_date"], axis=1)

In [27]:
# month of the year
fire_lulc_df['month'] = fire_lulc_df['month'].replace(to_replace=1, 
                                                        value="January")
fire_lulc_df['month'] = fire_lulc_df['month'].replace(to_replace=
                                                        2,
                                                        value="February")
fire_lulc_df['month'] = fire_lulc_df['month'].replace(to_replace=3, 
                                                value="March")
fire_lulc_df['month'] = fire_lulc_df['month'].replace(to_replace=4, 
                                                        value="April")
fire_lulc_df['month'] = fire_lulc_df['month'].replace(to_replace=5, 
                                                        value="May")
fire_lulc_df['month'] = fire_lulc_df['month'].replace(to_replace=6, 
                                                        value="June")
fire_lulc_df['month'] = fire_lulc_df['month'].replace(to_replace=7, 
                                                        value="July")
fire_lulc_df['month'] = fire_lulc_df['month'].replace(to_replace=8, 
                                                        value="August")
fire_lulc_df['month'] = fire_lulc_df['month'].replace(to_replace=9, 
                                                        value="September")
fire_lulc_df['month'] = fire_lulc_df['month'].replace(to_replace=10, 
                                                        value="October")
fire_lulc_df['month'] = fire_lulc_df['month'].replace(to_replace=11, 
                                                        value="November")
fire_lulc_df['month'] = fire_lulc_df['month'].replace(to_replace=12, 
                                                        value="December")

In [28]:
# converting to CSV file
fire_lulc_df.to_csv("../Datasets/fire_lulc_de.csv")

In [29]:
fire_lulc_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 18785 entries, 0 to 18784
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   brightness     18785 non-null  float64 
 1   confidence     18785 non-null  int64   
 2   bright_t31     18785 non-null  float64 
 3   frp            18785 non-null  float64 
 4   daynight       18785 non-null  object  
 5   lulc           18785 non-null  object  
 6   month          18785 non-null  object  
 7   period_day     18785 non-null  category
 8   plant_harvest  18785 non-null  object  
dtypes: category(1), float64(3), int64(1), object(4)
memory usage: 1.2+ MB


In [30]:
fire_lulc_df['month'] = fire_lulc_df.month.astype('category')
fire_lulc_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 18785 entries, 0 to 18784
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   brightness     18785 non-null  float64 
 1   confidence     18785 non-null  int64   
 2   bright_t31     18785 non-null  float64 
 3   frp            18785 non-null  float64 
 4   daynight       18785 non-null  object  
 5   lulc           18785 non-null  object  
 6   month          18785 non-null  category
 7   period_day     18785 non-null  category
 8   plant_harvest  18785 non-null  object  
dtypes: category(2), float64(3), int64(1), object(3)
memory usage: 1.0+ MB


In [31]:
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(fire_lulc_df)
categorical_columns = categorical_columns_selector(fire_lulc_df)

In [32]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

numerical_preprocessor = StandardScaler()
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")

In [33]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)])

In [34]:
from sklearn.cluster import KMeans
kmeans = KMeans(2)
clusters = kmeans.fit_predict(preprocessor)
labels = pd.DataFrame(clusters)
# labeledCustomers = pd.concat((fire_lulc_df,labels),axis=1)
# labeledCustomers = labeledCustomers.rename({0:'labels'},axis=1)

TypeError: float() argument must be a string or a real number, not 'ColumnTransformer'